## Home work


In [11]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-06-30 04:53:40--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 04:53:41 (18.7 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [12]:
import minsearch

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [13]:
index= minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [14]:
index.fit(documents)

In [21]:
#boost_dict el 3 indica que la question es 3 veces mas importante que text y section
query = "How do I execute a command in a running docker container?"

filter_dict = {"course": "data-engineering-zoomcamp"}
boost_dict = {"question": 3.0, "text": 1.0, "section": 1.0}

results = index.search(query, filter_dict, boost_dict, num_results= 5)

results

[{'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)\nVersion: 4.0.1\nHome: http://pgcli.com\nroot@pg-database:ny_taxi> \\dt\n+--------+------------------+-------+-------+\n| Schema | Name             | Type  | Owner |\n|--------+------------------+-------+-------|\n| public | yellow_taxi_data | table | root  |\n+--------+------------------+-------+-------+\nSELECT 1\nTime: 0.009s\nroot@pg-database:ny_taxi>',
  'section': 'Module 1: Docker and Terraform',
  'question': 'PGCLI - running in a Docker container',
 

In [22]:
!pip install openai

In [3]:
from openai import OpenAI

In [ ]:
import os

api_key = os.getenv('OPENAI_API_KEY')


In [4]:
client= OpenAI(api_key=api_key)

In [6]:
query= "How do I execute a command in a running docker container?"

In [7]:
response= client.chat.completions.create(
    model= 'gpt-4o',
    messages= [{"role": "user", "content": query}]
)

In [8]:
response.choices[0].message.content

"To execute a command in a running Docker container, you can use the `docker exec` command. This command allows you to run a new command in the context of an existing container. Here are the basic steps:\n\n1. **Identify the Container ID or Name:** First, you need to identify the container where you want to execute the command. You can do this with the `docker ps` command, which lists all running containers.\n\n    ```sh\n    docker ps\n    ```\n\n    This will output a list of running containers, including their IDs and names.\n\n2. **Execute the Command:** Once you have the container ID or name, you can use the `docker exec` command to run the desired command inside the container. The general syntax is:\n\n    ```sh\n    docker exec [OPTIONS] CONTAINER COMMAND [ARG...]\n    ```\n\n    - `OPTIONS`: You can use various options such as `-it` for interactive terminal.\n    - `CONTAINER`: The ID or name of the running container.\n    - `COMMAND`: The command you want to execute.\n    - `[

In [23]:
prompt_template= """
You're a teaching assistant. Answer the QUESTION based on the CONTEXT.
Use only the facts form the CONTEXT when answering the QUESTION.
If the CONTEXT don't have the answer, output NONE

QUESTION: {question}
CONTEXT: {context}

""".strip()

In [24]:
context= ""

for doc in results:
    context= context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [25]:
print(context)

section: Module 1: Docker and Terraform
question: PGCLI - running in a Docker container
answer: In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.
Below the usage with values used in the videos of the course for:
network name (docker network)
postgres related variables for pgcli
Hostname
Username
Port
Database name
$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1
175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi
Password for root:
Server: PostgreSQL 16.1 (Debian 16.1-1.pgdg120+1)
Version: 4.0.1
Home: http://pgcli.com
root@pg-database:ny_taxi> \dt
+--------+------------------+-------+-------+
| Schema | Name             | Type  | Owner |
|--------+------------------+-------+-------|
| public | yellow_taxi_data | table | root  |
+--------+------------------+-------+-------+
SELECT 1
Time: 0.009s
root@pg-database:ny_taxi>

section: Module 1: Docker and Terraform
qu

## Elasticsearch

In [26]:
from elasticsearch import Elasticsearch

In [27]:
es_client= Elasticsearch('http://localhost:9200')

In [28]:
es_client.info()

ObjectApiResponse({'name': '3c64b60885c6', 'cluster_name': 'docker-cluster', 'cluster_uuid': '2p2VNxtwRRK8q7zLdzJMyw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [30]:
index_settings= {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name= 'course-questions2'

es_client.indices.create(index= index_name, body= index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions2'})

In [31]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [32]:
from tqdm.auto import tqdm

In [33]:
for doc in tqdm(documents):
    es_client.index(index= index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [34]:
query= "How do I execute a command in a running docker container?"

In [64]:
#def elastic_search(query):
search_query= {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
response= es_client.search(index= index_name, body= search_query)
result_docs= []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
#return result_docs

In [65]:
result_docs

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [83]:
context_entries = [
    {
        "question": "How do I debug a docker container?",
        "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)"
    },
    {
        "question": "How do I copy files from my local machine to docker container?",
        "text": "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani"
    },
    {
        "question": "How do I copy files from a different folder into docker container’s working directory?",
        "text": "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nIn the Dockerfile, you can provide the folder containing the files that you want to copy over. The basic syntax is as follows:\nCOPY [\"src/predict.py\", \"models/xgb_model.bin\", \"./\"]\t\t\t\t\t\t\t\t\t\t\tGopakumar Gopinathan"
    },
    {
        "question": "The input device is not a TTY when running docker in interactive mode (Running Docker on Windows in GitBash)",
        "text": "$ docker exec -it 1e5a1b663052 bash\nthe input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'\nFix:\nwinpty docker exec -it 1e5a1b663052 bash\nA TTY is a terminal interface that supports escape sequences, moving the cursor around, etc.\nWinpty is a Windows software package providing an interface similar to a Unix pty-master for communicating with Windows console programs.\nMore info on terminal, shell, console applications hi and so on:\nhttps://conemu.github.io/en/TerminalVsShell.html\n(Marcos MJD)"
    },
    {
        "question": "How to fix error after running the Docker run command",
        "text": "Solution\nThis error was because there was another instance of gunicorn running. So I thought of removing this along with the zoomcamp_test image. However, it didn’t let me remove the orphan container. So I did the following\nRunning the following commands\ndocker ps -a <to list all docker containers>\ndocker images <to list images>\ndocker stop <container ID>\ndocker rm <container ID>\ndocker rmi image\nI rebuilt the Docker image, and ran it once again; this time it worked correctly and I was able to serve the test script to the endpoint."
    }
]

context = "\n\n".join([f"Q: {entry['question']}\nA: {entry['text']}" for entry in context_entries])


In [84]:
question = "How do I execute a command in a running docker container?"

prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

print(prompt)


You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [85]:
# prompt len
prompt_length = len(prompt)
print(prompt_length)

2714


In [86]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 775.1/775.1 kB 20.3 MB/s eta 0:00:00m eta 0:00:01


In [89]:
import tiktoken

In [90]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [96]:
# modify the prompt and calculate the number of tokens
tokenized_prompt = encoding.encode(prompt)
num_tokens = len(tokenized_prompt)
print(f"Number of tokens: {num_tokens}")

Number of tokens: 621


### Bonus: calculating the costs

In [97]:
cost= (num_tokens*0.005)/1000

In [99]:
print(f"Cost of tokens: {cost}$")

Cost of tokens: 0.003105$
